In [2]:
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings
import faiss
from langchain_community.document_loaders import PyPDFDirectoryLoader
from dotenv import load_dotenv
import os
from uuid import uuid4
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


In [3]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

data_dir = "data"
loader = PyPDFDirectoryLoader(data_dir)
documents = loader.load_and_split()
llm = ChatOpenAI(openai_api_key=api_key,model="gpt-3.5-turbo")

In [4]:

embeddings = OpenAIEmbeddings(openai_api_key=api_key)
index = faiss.IndexFlatL2(len(embeddings.embed_query("example query")))

# Create a mapping from index to document store IDs
index_to_docstore_id = {i: str(uuid4()) for i in range(len(documents))}
docstore = InMemoryDocstore({index_to_docstore_id[i]: doc for i, doc in enumerate(documents)})

vector_store = FAISS(embedding_function=embeddings, index=index, docstore=docstore, index_to_docstore_id=index_to_docstore_id)

In [5]:
vector_store.add_documents(documents)


['51311fb5-e74b-4db0-b242-1765c098d4a5',
 '177d165c-fbc7-4b07-952c-6d0a24e578c3',
 '58dd5198-d1bd-422e-bf4c-8b6f436303a3',
 'beed3e17-fa5b-414d-8f6c-b79689762fad',
 '05ae636d-902a-4546-8a5f-32998a6ab946',
 'c8c27612-a121-43dc-86fc-a9a8cf15af78',
 '2113166b-773f-4772-b36b-7c60bc8fd8e6',
 '98487dbf-f728-45c8-8477-ddfa80e776b5',
 '957e8d59-a014-4a78-80c6-9ee126796682',
 '9eb3d3fb-3ef3-4eaf-b733-cbd0ecd6c116',
 '3e4e7059-c372-4b3e-b4e9-5122fb0ed9a1',
 '8704efcf-ea91-4468-a161-7e0b65e32cbc',
 '03f30de8-2e59-466a-93c6-ea0ab994e7f9',
 'f77efff5-9945-4db0-ae25-0efee01b5e02',
 'b00b1a17-f30d-4875-9aa0-c0a58efbb9fc',
 '6c90183c-6d50-49fa-bc93-b1d8dc6fdcdf']

In [39]:
print("There are", len(vector_store.docstore._dict), "docs in the collection")
# Create the RAG chain
# rag_chain = RetrievalQA.from_llm(
#     llm=ChatOpenAI(openai_api_key=api_key),
#     retriever=vector_store.as_retriever(),
#     prompt=system_prompt_template,
#     return_source_documents=True,  # Ensure source documents are returned
#     verbose=True,
# )

There are 32 docs in the collection


In [16]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
from pprint import pprint

condense_question_system_template = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

condense_question_prompt = ChatPromptTemplate.from_messages([
    ("system", condense_question_system_template),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
])

history_aware_retriever = create_history_aware_retriever(
    llm, vector_store.as_retriever(), condense_question_prompt
)

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
])

qa_chain = create_stuff_documents_chain(llm, qa_prompt)
convo_qa_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

result = convo_qa_chain.invoke({"input": "Me de um resumo do artigo", "chat_history": []})
pprint(result)


{'answer': 'O artigo discute a aplicação da Lei com foco na justiça, '
           'reconhecendo que a perfeição da Lei é inatingível, mas defendendo '
           'que sua aplicação deve ser justa. Mostra visualizações de atenção '
           'em camadas específicas de um modelo, destacando como certas '
           'cabeças de atenção lidam com resolução de anáfora e estrutura '
           'frasal. Além disso, cita referências de trabalhos anteriores sobre '
           'processamento de linguagem natural e aprendizado de máquina.',
 'chat_history': [],
 'context': [Document(metadata={'source': 'data\\1706.03762v7.pdf', 'page': 13}, page_content='Input-Input Layer5\nThe\nLaw\nwill\nnever\nbe\nperfect\n,\nbut\nits\napplication\nshould\nbe\njust\n-\nthis\nis\nwhat\nwe\nare\nmissing\n,\nin\nmy\nopinion\n.\n<EOS>\n<pad>\nThe\nLaw\nwill\nnever\nbe\nperfect\n,\nbut\nits\napplication\nshould\nbe\njust\n-\nthis\nis\nwhat\nwe\nare\nmissing\n,\nin\nmy\nopinion\n.\n<EOS>\n<pad>\nInput-Input Layer5\

In [17]:
result["answer"]

'O artigo discute a aplicação da Lei com foco na justiça, reconhecendo que a perfeição da Lei é inatingível, mas defendendo que sua aplicação deve ser justa. Mostra visualizações de atenção em camadas específicas de um modelo, destacando como certas cabeças de atenção lidam com resolução de anáfora e estrutura frasal. Além disso, cita referências de trabalhos anteriores sobre processamento de linguagem natural e aprendizado de máquina.'

In [82]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Defina o template do prompt
prompt = PromptTemplate.from_template(
    "Você é um assistente que responde apenas com informações coletadas dos documentos. "
    "Contexto:\n{context}\n\nPergunta: {question}"
)

# Função para formatar documentos
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Crie a cadeia RAG
context_runnable = vector_store.as_retriever() | format_docs
question_runnable = RunnablePassthrough()

# Crie uma sequência de Runnable com pelo menos dois passos
rag_chain = (
    RunnableSequence(
        first=context_runnable,
        middle=question_runnable,
        last=prompt
    )
    | ChatOpenAI(openai_api_key=api_key)
    | StrOutputParser()
)

# Defina sua consulta
query = "Me de um resumo do artigo"

# Chame a cadeia RAG
result = rag_chain.invoke({'question': query})

# Acesse os resultados
answer = result['result']  # A resposta gerada
source_documents = result['source_documents']  # Os documentos recuperados

# Imprima os resultados
print("Answer:", answer)
print("Source Documents:", source_documents)

TypeError: can only concatenate list (not "RunnablePassthrough") to list